<a href="https://colab.research.google.com/github/samirjhb/samirjhb/blob/main/geomovilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
gdrive='/content/gdrive/MyDrive/Classroom/BIG DATA'

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install pyspark
!pip install pyarrow


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]")\
      .appName("quadrant").getOrCreate()

In [ ]:
QD_df=gdrive+'/5a66484a-d842-47a7-8527-f755e39b4a6e.csv'
df_QD=spark.read.csv(QD_df,inferSchema=True,sep=",",header=True)
df_QD.createOrReplaceTempView('df_QD')

In [ ]:
!mkdir streaming


In [ ]:
query=spark.sql('select device_id,count(*) \
from df_QD group by device_id order \
by count(*) desc limit 100' )
id=query.toPandas().device_id[1]
df_id=spark.sql('''select latitude,longitude,timestamp \
from df_QD where device_id='$id' '''.replace('$id',id)).toPandas()
df_id=df_id.sort_values('timestamp')
df_id.to_csv('data_qd.csv')

In [ ]:
%%python --bg
import time
import numpy as np
import pandas as pd
df_id=pd.read_csv('data_qd.csv')
for n in range(0,int(len(df_id)/200)):
     df_id.iloc[n*200:((n+1)*200)-1]\
     .to_csv('/content/streaming/data'+str(n)+'.csv',index=False)
     time.sleep(5)

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
      .master('local')\
      .appName('QD_streaming')\
      .config('spark.ui.port',4050).getOrCreate()
#####
from pyspark.sql.types import *
schema=StructType([StructField('id',StringType(),True),
                   StructField('latitude',StringType(),True),
                   StructField('longitude',StringType(),True),
                   StructField('timestamp',StringType(),True),])
df=spark.readStream\
  .format('csv').schema(schema)\
  .option('header',True).load('streaming')

query=df.writeStream.outputMode('update')\
      .queryName('tabla')\
      .format('memory')\
      .start()



In [ ]:
spark.stop()

In [ ]:
data=[]
for i in range(0,100):
      data.append(spark.sql("select latitude,longitude,timestamp from tabla order by timestamp desc limit 1")\
      .toPandas().values[0].tolist())

In [ ]:
data